In [1]:
import numpy as np
import re

from tensorflow.keras.layers import Dense, GRU, Input, Dropout, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [11]:
texts_true = [
    "Чудова новина!",
    "Нарешті вийшов довгоочікуваний фільм, який ви так хотіли побачити.",
    "Збирайте друзів, купуйте попкорн і вирушайте в кінотеатр на незабутні враження!"
]

texts_false = [
    "Неприємна ситуація.",
    "Застряг у пробці на годину, через що спізнився на важливу зустріч.",
    "Настрій зіпсований, плани порушені, а попереду ще багато справ."
]

texts = texts_true + texts_false
count_true = len(texts_true)
count_false = len(texts_false)
total_lines = count_true + count_false
print(count_true, count_false, total_lines)


maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(texts)

dist = list(tokenizer.word_counts.items())
print(dist[:10])
print(texts[0][:100])


max_text_len = 10
data = tokenizer.texts_to_sequences(texts)
data_pad = pad_sequences(data, maxlen=max_text_len)
print(data_pad)

print( list(tokenizer.word_index.items()) )


X = data_pad
Y = np.array([[1, 0]]*count_true + [[0, 1]]*count_false)
print(X.shape, Y.shape)

indeces = np.random.choice(X.shape[0], size=X.shape[0], replace=False)
X = X[indeces]
Y = Y[indeces]

3 3 6
[('чудова', 1), ('новина', 1), ('нарешті', 1), ('вийшов', 1), ('довгоочікуваний', 1), ('фільм', 1), ('який', 1), ('ви', 1), ('так', 1), ('хотіли', 1)]
Чудова новина!
[[ 0  0  0  0  0  0  0  0  2  3]
 [ 0  4  5  6  7  8  9 10 11 12]
 [14 15 16 17 18 19 20  1 21 22]
 [ 0  0  0  0  0  0  0  0 23 24]
 [26 27  1 28 29 30 31  1 32 33]
 [ 0 34 35 36 37 38 39 40 41 42]]
[('на', 1), ('чудова', 2), ('новина', 3), ('нарешті', 4), ('вийшов', 5), ('довгоочікуваний', 6), ('фільм', 7), ('який', 8), ('ви', 9), ('так', 10), ('хотіли', 11), ('побачити', 12), ('збирайте', 13), ('друзів', 14), ('купуйте', 15), ('попкорн', 16), ('і', 17), ('вирушайте', 18), ('в', 19), ('кінотеатр', 20), ('незабутні', 21), ('враження', 22), ('неприємна', 23), ('ситуація', 24), ('застряг', 25), ('у', 26), ('пробці', 27), ('годину', 28), ('через', 29), ('що', 30), ('спізнився', 31), ('важливу', 32), ('зустріч', 33), ('настрій', 34), ('зіпсований', 35), ('плани', 36), ('порушені', 37), ('а', 38), ('попереду', 39), ('ще',

In [12]:
model = Sequential()
model.add(Embedding(maxWordsCount, 128, input_length = max_text_len))
model.add(GRU(128, return_sequences=True))
model.add(GRU(64))
model.add(Dense(2, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))

history = model.fit(X, Y, batch_size=32, epochs=50)

reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

def sequence_to_text(list_of_indices):
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 10, 128)           128000    
                                                                 
 gru_6 (GRU)                 (None, 10, 128)           99072     
                                                                 
 gru_7 (GRU)                 (None, 64)                37248     
                                                                 
 dense_3 (Dense)             (None, 2)                 130       
                                                                 
Total params: 264450 (1.01 MB)
Trainable params: 264450 (1.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 0.6929 - accuracy: 0.3333
Epoch 2/50
1/1 [==============================] - 0s 8m

In [21]:
t = "Чудова новина".lower()
data = tokenizer.texts_to_sequences([t])
data_pad = pad_sequences(data, maxlen=max_text_len)
print( sequence_to_text(data[0]) )

res = model.predict(data_pad)

['чудова', 'новина']
1/1 [==============================] - 0s 14ms/step


In [25]:
if np.argmax(res) == 0:
    print("Результат. Текст позитивний.")
else:
    print("Результат. Текст негативний.")

Результат. Текст позитивний.


In [23]:
t1 = "Неприємна ситуація".lower()
data1 = tokenizer.texts_to_sequences([t1])
data1_pad = pad_sequences(data1, maxlen=max_text_len)
print( sequence_to_text(data1[0]) )

res1 = model.predict(data1_pad)

['неприємна', 'ситуація']
1/1 [==============================] - 0s 14ms/step


In [27]:
if np.argmax(res1) == 0:
    print("Результат. Текст позитивний.")
else:
    print("Результат. Текст негативний.")

Результат. Текст негативний.
